
# Stage 10a — Modeling: Linear Regression (Muideenn)

This notebook fits a baseline Linear Regression model, runs residual diagnostics, and reports **R²** and **RMSE**. It uses your cleaned dataset if present at `data/processed/clean.csv` (expects numeric target column `y` and numeric features); otherwise it generates a synthetic dataset for prototyping.

**Reproducibility:** A fixed random seed is used where applicable.


In [ ]:

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Local utils
from src.utils import project_path, load_csv, save_df

# Plotting defaults
plt.rcParams['figure.figsize'] = (7.5, 5)
plt.rcParams['axes.grid'] = True

RNG_SEED = 42
np.random.seed(RNG_SEED)
print('Setup complete.')



## 1) Load cleaned dataset (if present) or generate synthetic data
Expected schema if using your data:
- `y`: numeric target
- Other numeric columns are treated as features.


In [ ]:

clean_csv = project_path('data/processed/clean.csv')
if clean_csv.exists():
    df = load_csv('data/processed/clean.csv')
    # Heuristic: require a 'y' column + at least 1 numeric feature column
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    assert 'y' in df.columns, "Your clean.csv must have a numeric target column named 'y'."
    X_cols = [c for c in num_cols if c != 'y']
    assert len(X_cols) >= 1, "Your clean.csv must include at least one numeric feature column besides 'y'."
    print(f"Loaded real dataset with shape {df.shape}. Using features: {X_cols}")
else:
    # Synthetic data: y = 3*x1 - 2*x2 + noise; plus a nonlinear relation in x3 to stress diagnostics
    n = 600
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.uniform(-2, 2, n)
    eps = np.random.normal(0, 1.0, n)
    y = 3*x1 - 2*x2 + 0.5*(x3**2) + eps  # include a quadratic effect in x3 to test linearity assumption
    
    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})
    save_df(df, 'data/processed/clean.csv')  # save for reference
    X_cols = ['x1','x2','x3']
    print(f"Generated synthetic dataset with shape {df.shape}. Using features: {X_cols}")
    
df.head()



## 2) Train/Test split
We use an 80/20 split with a fixed random state for reproducibility.


In [ ]:

X = df[X_cols].copy()
y = df['y'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RNG_SEED)
X_train.shape, X_test.shape



## 3) Fit `sklearn.linear_model.LinearRegression`


In [ ]:

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_train = linreg.predict(X_train)
y_pred_test  = linreg.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test  = r2_score(y_test, y_pred_test)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test  = mean_squared_error(y_test, y_pred_test, squared=False)

print('Coefficients:', dict(zip(X_cols, linreg.coef_)))
print('Intercept   :', linreg.intercept_)
print(f'R^2 train={r2_train:.3f}, test={r2_test:.3f}')
print(f'RMSE train={rmse_train:.3f}, test={rmse_test:.3f}')



## 4) Predictions & residuals


In [ ]:

resid_train = y_train - y_pred_train
resid_test  = y_test  - y_pred_test

res_df = pd.DataFrame({
    'y_train': y_train, 'y_pred_train': y_pred_train, 'resid_train': resid_train
})
res_df.head()



## 5) Diagnostics & Plots
We check:
- **Residuals vs Fitted** (linearity & homoscedasticity)
- **Residual Histogram** (normality rough check)
- **QQ Plot** (normality diagnostic)
- **Residuals vs Key Predictor** (linearity wrt a feature)
- **Residual Lag‑1** (autocorrelation hint for independence)


In [ ]:

# Residuals vs Fitted (Train)
plt.figure()
plt.scatter(y_pred_train, resid_train, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.xlabel('Fitted (Train)')
plt.ylabel('Residuals (Train)')
plt.title('Residuals vs Fitted (Train)')
plt.show()

# Histogram of residuals (Train)
plt.figure()
plt.hist(resid_train, bins=30)
plt.xlabel('Residual')
plt.ylabel('Count')
plt.title('Residual Histogram (Train)')
plt.show()

# QQ plot using scipy
plt.figure()
stats.probplot(resid_train, dist="norm", plot=plt)
plt.title('QQ Plot of Residuals (Train)')
plt.show()

# Residuals vs Key Predictor (use first column)
key_col = X_cols[0]
plt.figure()
plt.scatter(X_train[key_col], resid_train, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.xlabel(f'{key_col} (Train)')
plt.ylabel('Residuals (Train)')
plt.title(f'Residuals vs {key_col} (Train)')
plt.show()

# Residual lag-1 (Train)
resid_sorted = pd.Series(resid_train).reset_index(drop=True)
lag1 = resid_sorted.shift(1)
plt.figure()
plt.scatter(lag1[1:], resid_sorted[1:], alpha=0.6)
plt.axhline(0, linestyle='--')
plt.xlabel('Residual (t-1)')
plt.ylabel('Residual (t)')
plt.title('Residual Lag-1 (Train)')
plt.show()



## 6) Interpretation of Assumptions
**Linearity:** Examine **Residuals vs Fitted** and **Residuals vs key predictor**. If we see curvature (e.g., U‑shape), that suggests a missing nonlinear term (like a squared feature).  
**Independence:** If the **Residual lag‑1** scatter shows structure (e.g., diagonal pattern), residuals may be autocorrelated (time‑series effects).  
**Homoscedasticity:** Look for constant variance across fitted values. A clear funnel shape indicates heteroscedasticity.  
**Normality:** The **Histogram** and **QQ plot** help see heavy tails or skew. Mild deviations may be fine for unbiased estimates of coefficients; prediction intervals may be affected.



## 7) (Optional) Simple Transformation: add a squared term
Adding a polynomial term (e.g., `x3^2`) keeps the model **linear in parameters**, so it's still linear regression.


In [ ]:

from sklearn.preprocessing import PolynomialFeatures

poly_cols = X_cols.copy()
augment_col = None
if 'x3' in X_cols:
    augment_col = 'x3'
elif len(X_cols) >= 1:
    augment_col = X_cols[-1]

if augment_col is not None:
    X_train_aug = X_train.copy()
    X_test_aug  = X_test.copy()
    X_train_aug[f'{augment_col}^2'] = X_train_aug[augment_col]**2
    X_test_aug[f'{augment_col}^2']  = X_test_aug[augment_col]**2

    lin2 = LinearRegression().fit(X_train_aug, y_train)
    y_pred_train2 = lin2.predict(X_train_aug)
    y_pred_test2  = lin2.predict(X_test_aug)

    r2_train2 = r2_score(y_train, y_pred_train2)
    r2_test2  = r2_score(y_test, y_pred_test2)
    rmse_train2 = mean_squared_error(y_train, y_pred_train2, squared=False)
    rmse_test2  = mean_squared_error(y_test, y_pred_test2, squared=False)

    print('Augmented coefficients:', dict(zip(X_train_aug.columns, lin2.coef_)))
    print('Intercept             :', lin2.intercept_)
    print(f'R^2 train={r2_train2:.3f}, test={r2_test2:.3f}')
    print(f'RMSE train={rmse_train2:.3f}, test={rmse_test2:.3f}')
else:
    print("No numeric feature available to square.")



## 8) Conclusion
- Report **R²** and **RMSE** on the test set and whether they meet your use‑case needs (prediction vs. explanation).  
- If diagnostics show curvature or heteroscedasticity, consider transformations, interaction terms, or a different model (e.g., regularized regression, tree‑based models).  
- Document trust level and next steps.
